Check Tide Map
==============

Check if a given point is within a tide model domain

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  
- https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/
- ftp://ftp.esr.org/pub/datasets/tmd/  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  

Finite Element Solution (FES) provided by AVISO  
- https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html

#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  
 - [ipyleaflet: Jupyter / Leaflet bridge enabling interactive maps](https://github.com/jupyter-widgets/ipyleaflet)  

#### Program Dependencies

- `convert_ll_xy.py`: convert lat/lon points to and from projected coordinates  
- `model.py`: retrieves tide model parameters for named tide models
- `read_tide_model.py`: extract tidal harmonic constants from OTIS tide models  
- `read_netcdf_model.py`: extract tidal harmonic constants from netcdf models  
- `read_GOT_model.py`: extract tidal harmonic constants from GSFC GOT models  
- `read_FES_model.py`: extract tidal harmonic constants from FES tide models  

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
The widgets can be installed as described below.  
```
pip3 install --user ipywidgets
jupyter nbextension install --user --py widgetsnbextension
jupyter nbextension enable --user --py widgetsnbextension
jupyter-notebook
```

#### Load modules

In [ ]:
from __future__ import print_function

import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipyleaflet as leaflet

import pyTMD.time
import pyTMD.model
import pyTMD.read_tide_model
import pyTMD.read_netcdf_model
import pyTMD.read_GOT_model
import pyTMD.read_FES_model
from pyTMD.spatial import wrap_longitudes
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# set the directory with tide models
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)

# dropdown menu for setting tide model
model_list = ['CATS0201','CATS2008','TPXO9-atlas','TPXO9-atlas-v2',
    'TPXO9-atlas-v3','TPXO9-atlas-v4','TPXO9-atlas-v5','TPXO9.1',
    'TPXO8-atlas','TPXO7.2','AODTM-5','AOTIM-5','AOTIM-5-2018','Arc2kmTM',
    'Gr1km-v2','GOT4.7','GOT4.8','GOT4.10','FES2014','EOT20']
modelDropdown = widgets.Dropdown(
    options=model_list,
    value='GOT4.10',
    description='Model:',
    disabled=False,
)

# dropdown menu for setting ATLAS format model
atlas_list = ['OTIS','netcdf']
atlasDropdown = widgets.Dropdown(
    options=atlas_list,
    value='netcdf',
    description='ATLAS:',
    disabled=False,
)

# checkbox for setting if tide files are compressed
compressCheckBox = widgets.Checkbox(
    value=True,
    description='Compressed?',
    disabled=False,
)

# display widgets for setting directory and model
widgets.VBox([
    dirText,
    modelDropdown,
    atlasDropdown,
    compressCheckBox
])

In [ ]:
# default coordinates to use
LAT,LON = (32.86710263,-117.25750387)
m = leaflet.Map(center=(LAT,LON), zoom=12, basemap=leaflet.basemaps.Esri.WorldTopoMap)
# add control for zoom
zoom_slider = widgets.IntSlider(description='Zoom level:', min=0, max=15, value=7)
widgets.jslink((zoom_slider, 'value'), (m, 'zoom'))
zoom_control =  leaflet.WidgetControl(widget=zoom_slider, position='topright')
m.add_control(zoom_control)
# add marker with default location
marker = leaflet.Marker(location=(LAT,LON), draggable=True)
m.add_layer(marker)
# add text with marker location
markerText = widgets.Text(
    value='{0:0.8f},{1:0.8f}'.format(LAT,LON),
    description='Lat/Lon:',
    disabled=False
)

# add function for setting marker text if location changed
def set_marker_text(sender):
    LAT,LON = marker.location
    markerText.value = '{0:0.8f},{1:0.8f}'.format(LAT,wrap_longitudes(LON))
# add function for setting map center if location changed
def set_map_center(sender):
    m.center = marker.location
# add function for setting marker location if text changed
def set_marker_location(sender):
    LAT,LON = [float(i) for i in markerText.value.split(',')]
    marker.location = (LAT,LON)
    
# watch marker widgets for changes
marker.observe(set_marker_text)
markerText.observe(set_marker_location)
m.observe(set_map_center)
# add control for marker location
marker_control =  leaflet.WidgetControl(widget=markerText, position='bottomright')
m.add_control(marker_control)
m

In [ ]:
# leaflet location
LAT,LON = marker.location
# verify longitudes
LON = wrap_longitudes(LON)

# get model parameters
model = pyTMD.model(dirText.value, format=atlasDropdown.value,
    compressed=compressCheckBox.value).elevation(modelDropdown.value)

# adjust dimensions of input coordinates to be iterable
LON = np.atleast_1d(LON)
LAT = np.atleast_1d(LAT)
    
# read tidal constants and interpolate to grid points
if model.format in ('OTIS','ATLAS'):
    # if reading a single OTIS solution
    xi,yi,hz,mz,iob,dt = pyTMD.read_tide_model.read_tide_grid(model.grid_file)
    # adjust dimensions of input coordinates to be iterable
    # run wrapper function to convert coordinate systems of input lat/lon
    x,y = pyTMD.convert_ll_xy(np.atleast_1d(LON),np.atleast_1d(LAT),
        model.projection,'F')
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    if (np.min(x) < np.min(xi)) & (model.projection == '4326'):
        lt0, = np.nonzero(x < 0)
        x[lt0] += 360.0
    if (np.max(x) > np.max(xi)) & (model.projection == '4326'):
        gt180, = np.nonzero(x > 180)
        x[gt180] -= 360.0
elif (model.format == 'netcdf'):
    # if reading a netCDF OTIS atlas solution
    xi,yi,hz = pyTMD.read_netcdf_model.read_netcdf_grid(model.grid_file,
        GZIP=model.compressed, TYPE='z')
    # invert bathymetry mask
    mz = np.invert(hz.mask)
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    x,y = np.copy([LON,LAT]).astype(np.float64)
    lt0, = np.nonzero(x < 0)
    x[lt0] += 360.0
elif (model.format == 'GOT'):
    # if reading a NASA GOT solution
    hc,xi,yi,c = pyTMD.read_GOT_model.read_GOT_grid(model.model_file[0],
        GZIP=model.compressed)
    # invert tidal constituent mask
    mz = np.invert(hc.mask)
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    x,y = np.copy([LON,LAT]).astype(np.float64)
    lt0, = np.nonzero(x < 0)
    x[lt0] += 360.0
elif (model.format == 'FES'):
    # if reading a FES netCDF solution
    hc,xi,yi = pyTMD.read_FES_model.read_netcdf_file(model.model_file[0],
        GZIP=model.compressed, TYPE='z', VERSION=model.version)
    # invert tidal constituent mask
    mz = np.invert(hc.mask)
    # adjust longitudinal convention of input latitude and longitude
    # to fit tide model convention
    x,y = np.copy([LON,LAT]).astype(np.float64)
    lt0, = np.nonzero(x < 0)
    x[lt0] += 360.0

# check coordinates on tide grid
fig,ax = plt.subplots(num=1,figsize=(12,12), dpi=200)
ax.imshow(mz, interpolation='nearest',
    extent=(xi.min(),xi.max(),yi.min(),yi.max()),
    origin='lower', cmap='gray')
ax.plot(x,y,'r*')
# no ticks on the x and y axes
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])
# stronger linewidth on frame
[i.set_linewidth(2.0) for i in ax.spines.values()]
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()